## Notebook introduction

This notebook gets data from **www.worldbank.org** and **www.interest.co.nz** by using Julia. Used some user-defined functions to obtain data.

The dataset obtained from **www.interest.co.nz** using unofficial API:
- usd-nzd_exchange_rate.csv

In [1]:
import Pkg

In [2]:
using Pkg

In [3]:
Pkg.add("JSON")
Pkg.add("HTTP")
Pkg.add("FileIO")
Pkg.add("WorldBankData")    # DataBank is an analysis and visualisation tool that contains collections of time series data on a variety of topics.

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = RequestError: HTTP/2 502 while requesting https://pkg.julialang.org/registries
└ @ Pkg.Registry /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Pkg/src/Registry/Registry.jl:69
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [4]:
using HTTP
using JSON
using Dates
using DataFrames
using CSV

In [5]:
current_directory = @__DIR__
data_file_directory = joinpath(current_directory, "DataSet")
isdir(data_file_directory) || mkdir(data_file_directory)  # Create the directory if it does not exist

true

### 1. Explore **www.worldbank.org**

#### Enable WorldBankData to query and download interesting datasets

In [6]:
using WorldBankData

#### Query using methods provided in WorldBankData API

1. Query data sets related to **CPI**

In [7]:
CPI_WDI = search_wdi("indicators","description",r"Consumer Price Index"i)

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
1,CPTOTNSXN,"CPI Price, nominal",The consumer price index reflects the change in prices for the average consumer of a constant basket of consumer goods. Data is not seasonally adjusted.,Global Economic Monitor,World Bank staff calculations based on Datastream data.
2,CPTOTSAXN,"CPI Price, nominal, seas. adj.",The consumer price index reflects the change in prices for the average consumer of a constant basket of consumer goods. Data is in nominal terms and seasonally adjusted.,Global Economic Monitor,World Bank staff calculations based on Datastream data.
3,CPTOTSAXNZGY,"CPI Price, % y-o-y, nominal, seas. adj.","The consumer price index reflects the change in prices for the average consumer of a constant basket of consumer goods. Data is in nominal percentage terms, measured on a year-on-year basis, and seasonally adjusted.",Global Economic Monitor,World Bank staff calculations based on Datastream data.
4,FP.CPI.TOTL,Consumer price index (2010 = 100),"Consumer price index reflects changes in the cost to the average consumer of acquiring a basket of goods and services that may be fixed or changed at specified intervals, such as yearly. The Laspeyres formula is generally used. Data are period averages.",World Development Indicators,"International Monetary Fund, International Financial Statistics and data files."
5,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)","Inflation as measured by the consumer price index reflects the annual percentage change in the cost to the average consumer of acquiring a basket of goods and services that may be fixed or changed at specified intervals, such as yearly. The Laspeyres formula is generally used.",World Development Indicators,"International Monetary Fund, International Financial Statistics and data files."
6,FP.FPI.TOTL,Food price index (2000 = 100),Food price index is a subindex of the consumer price index.,WDI Database Archives,"United Nations, Statistical Yearbook and Monthly Bulletin of Statistics."
7,MO.INDEX.ECON.XQ,Sustainable Economic Opportunity,"Public Management: Within this subcategory the Ibrahim Index measures: (i) Quality of Public Administration – clustered indicator (average) of variables from the African Development Bank and the World Bank measuring the extent to which the civil service is structured to effectively and ethically design policy and deliver services. (ii) Quality of Budget Management – clustered indicator (average) of variables from the African Development Bank and the World Bank measuring the extent to which there is a comprehensive and credible budget, linked to policy priorities, with mechanisms to ensure implementation and reporting. (iii) Currency Inside Banks – total stock of currency held within banks as a proportion of the money supply in an economy (OD). (iv) Ratio of Total Revenue to Total Expenditure – total budget revenue as a proportion of total budget expenditure (OD). (v) Ratio of Budget Deficit or Surplus to GDP – budget deficit or budget surplus as a proportion of Gross Domestic Product. (vi) Management of Public Debt – clustered indicator (average) of variables from the African Development Bank and the World Bank measuring short- and medium term sustainability of fiscal policy and its impact on growth. (vii) Inflation – annual average change in the consumer price index. (viii) Ratio of External Debt Service to Exports – total external debt service due, expressed as a proportion of exports of goods, non-factor services, income and workers’ remittances. (ix) Imports Covered by Reserves – period of time that imports could be paid for by foreign exchange reserves. (x) Statistical Capacity – national statistical systems and their adherence to international norms in the areas of: Methodology (of compiling statistics and indicators); Regularity and coverage of censuses and surveys; Regularity, timeliness and accessibility of key socioeconomi

By looking at the data frame, the second data set is what we want. This data set was calculated by World Bank staff based on Datastream data. The consumer price index reflects changes in prices paid by ordinary consumers for a fixed basket of consumer goods. We selected this data set to eliminate seasonality. Seasonality in different country regions can bias index statistics.

In [8]:
choose_row_CPI = CPI_WDI[5, :]

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
5,FP.CPI.TOTL.ZG,"Inflation, consumer prices (annual %)","Inflation as measured by the consumer price index reflects the annual percentage change in the cost to the average consumer of acquiring a basket of goods and services that may be fixed or changed at specified intervals, such as yearly. The Laspeyres formula is generally used.",World Development Indicators,"International Monetary Fund, International Financial Statistics and data files."


In [9]:
Indicator_CPI = CPI_WDI[5, :indicator]
println("The indicator of the data set we are interested in is: $(Indicator_CPI) ")

The indicator of the data set we are interested in is: FP.CPI.TOTL.ZG 


2. Query data sets related to **Unemployment Rate**

In [10]:
UNRATE_WDI = search_wdi("indicators","description",r"unemployment"i)

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
1,fin37.38.t.d,Received government transfer or pension (% age 15+),"The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector.",Global Financial Inclusion,Global Findex database
2,fin37.38.t.d.1,"Received government transfer or pension, female (% age 15+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, female (% age 15+)",Global Financial Inclusion,Global Findex database
3,fin37.38.t.d.10,"Received government transfer or pension, urban (% age 15+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, urban (% age 15+)",Global Financial Inclusion,Global Findex database
4,fin37.38.t.d.11,"Received government transfer or pension, out of labor force (% age 15+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, out of labor force (% age 15+)",Global Financial Inclusion,Global Findex database
5,fin37.38.t.d.12,"Received government transfer or pension, in labor force (% age 15+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, in labor force (% age 15+)",Global Financial Inclusion,Global Findex database
6,fin37.38.t.d.2,"Received government transfer or pension, male (% age 15+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, male (% age 15+)",Global Financial Inclusion,Global Findex database
7,fin37.38.t.d.3,"Received government transfer or pension, young (% ages 15-24)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or public sector, young (% ages 15-24)",Global Financial Inclusion,Global Findex database
8,fin37.38.t.d.4,"Received government transfer or pension, older (% age 25+)","The percentage of respondents who report personally receiving any transfer or pension from the government in the past year. This includes payments for educational or medical expenses, unemployment benefits, subsidy payments, or any kind of social benefits. It also includes payments for a pension from the government, military, or publi

In [11]:
choose_row_UNRATE = UNRATE_WDI[76, :]

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
76,SL.UEM.TOTL.ZS,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Unemployment refers to the share of the labor force that is without work but available for and seeking employment.,World Development Indicators,"International Labour Organization. “ILO Modelled Estimates and Projections database (ILOEST)” ILOSTAT. Accessed September 05, 2023. https://ilostat.ilo.org/data/."


The 75th result of the search is what we are interested in. The relevant introduction of the data set can be seen from the data frame above. It comes from the International Labor Organization. It collects the proportion of the labor force that is not working but has a job to find and is looking for a job , the definitions of labor force and unemployment vary from country to country.

In [12]:
Indicator_UNRATE = UNRATE_WDI[76, :indicator]
println("The indicator of the data set we are interested in is: $(Indicator_UNRATE) ")

The indicator of the data set we are interested in is: SL.UEM.TOTL.ZS 


3. Query data sets related to **Foreign Investment**

In [13]:
Foreign_Investment_WDI = search_wdi("indicators","name",r"foreign direct investment"i)

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
1,BG.KLT.DINV.GD.PP.ZS,"Gross foreign direct investment (% of GDP, PPP)",,WDI Database Archives,
2,BG.KLT.DINV.GD.ZS,Gross foreign direct investment (% of GDP),,WDI Database Archives,
3,BM.KLT.DINV.CD,"Foreign direct investment, net outflows by reporting economy (IMF-BoP, current US$)","Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows net outflows of investment from the reporting economy to the rest of the world. Data are in current U.S. dollars.",Africa Development Indicators,"International Monetary Fund, Balance of Payments Statistics Yearbook and data files."
4,BM.KLT.DINV.CD.WD,"Foreign direct investment, net outflows (BoP, current US$)","Foreign direct investment refers to direct investment equity flows in an economy. It is the sum of equity capital, reinvestment of earnings, and other capital. Direct investment is a category of cross-border investment associated with a resident in one economy having control or a significant degree of influence on the management of an enterprise that is resident in another economy. Ownership of 10 percent or more of the ordinary shares of voting stock is the criterion for determining the existence of a direct investment relationship. This series shows net outflows of investment from the reporting economy to the rest of the world. Data are in current U.S. dollars.",World Development Indicators,"International Monetary Fund, Balance of Payments database, supplemented by data from the United Nations Conference on Trade and Development and official national sources."
5,BM.KLT.DINV.GD.ZS,"Foreign direct investment, net outflows (% of GDP)","Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows net outflows of investment from the reporting economy to the rest of the world and is divided by GDP.",WDI Database Archives,"International Monetary Fund, International Financial Statistics and Balance of Payments databases, World Bank, International Debt Statistics, and World Bank and OECD GDP estimates."
6,BM.KLT.DINV.WD.GD.ZS,"Foreign direct investment, net outflows (% of GDP)","Foreign direct investment refers to direct investment equity flows in an economy. It is the sum of equity capital, reinvestment of earnings, and other capital. Direct investment is a category of cross-border investment associated with a resident in one economy having control or a significant degree of influence on the management of an enterprise that is resident in another economy. Ownership of 10 percent or more of the ordinary shares of voting stock is the criterion for determining the existence of a direct investment relationship. This series shows net outflows of investment from the reporting economy to the rest of the world, and is divided by GDP.",World Development Indicators,"International Monetary Fund, Balance of Payments database, supplemented by data from the United Nations Conference on Trade and Development and official national sources."
7,BN.KLT.DINV.CD,"Foreign direct investment, net (BoP, current US$)","Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other lo

In [14]:
choose_row_UNRATE = Foreign_Investment_WDI[7, :]

Row,indicator,name,description,source_database,source_organization
,String,String,String,String,String
7,BN.KLT.DINV.CD,"Foreign direct investment, net (BoP, current US$)","Foreign direct investment are the net inflows of investment to acquire a lasting management interest (10 percent or more of voting stock) in an enterprise operating in an economy other than that of the investor. It is the sum of equity capital, reinvestment of earnings, other long-term capital, and short-term capital as shown in the balance of payments. This series shows total net FDI. In BPM6, financial account balances are calculated as the change in assets minus the change in liabilities. Net FDI outflows are assets and net FDI inflows are liabilities. Data are in current U.S. dollars.",World Development Indicators,"International Monetary Fund, Balance of Payments Statistics Yearbook and data files."


In [15]:
Indicator_Foreign_Investment = Foreign_Investment_WDI[7, :indicator]
println("The indicator of the data set we are interested in is: $(Indicator_Foreign_Investment) ")

The indicator of the data set we are interested in is: BN.KLT.DINV.CD 


### 2. Explore **www.interest.co.nz**

#### Define related functions

In [16]:
function save_csv(data::DataFrame, filename)
    # Ensure the "dataset" subdirectory exists, or create it
    if !isdir(joinpath(current_directory, "DataSet"))
        mkdir(joinpath(current_directory, "DataSet"))
    end
    
    path = joinpath(current_directory, "DataSet", filename)  
    CSV.write(path, data)
end

function get_json(id::String)
    resp = HTTP.request("POST", "https://www.interest.co.nz/chart-data/get-csv-data";
        headers = Dict("Content-Type" => "application/x-www-form-urlencoded; charset=utf-8"),
        body = HTTP.URIs.escapeuri(Dict("nids[]" => id)))

    json_body = JSON.parse(transcode(String, resp.body))
    return json_body
end

function json2dataframe(id::String, json, colname)
    lines = json[id]["csv_data"][1]

    df = DataFrame(date = String[], data = Float64[])

    for line in lines
        t = unix2datetime(line[1] / 1000)
        ts = Dates.format(t, "Y-m-d")
        d = 1 / line[2]  # Take the inverse of the exchange rate
        push!(df, [ts, d])
    end

    rename!(df,:data => colname)
end

function get_save_csv(id::String, filename, colname)
    json = get_json(id)
    df = json2dataframe(id, json, colname)
    save_csv(df, filename)
end

get_save_csv (generic function with 1 method)

#### Get a time series of New Zealand exchange rates

In [17]:
# Get usd-nzd exchange rate
get_save_csv("79333", "nz_exchange_rate.csv", "NZD exchange rate")

"/Users/linsong/Documents/Data 422 Group Assignment/DataSet/nz_exchange_rate.csv"